In [1]:
from music21 import converter, instrument, note, chord
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

Using TensorFlow backend.


In [2]:
notes = []

music = converter.parse('./test1.mid')

parts = instrument.partitionByInstrument(music)

In [3]:
if parts:
    notes_to_parse = parts.parts[0].recurse()
else:
    notes_to_parse = music.flat.notes

In [4]:
for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

In [5]:
# notes.insert(0, 'start')
notes

['E-3',
 'G#2',
 'C2',
 'F#2',
 'E-3',
 'G#2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'C#3',
 'G#2',
 'F#2',
 'C#3',
 'G#2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'D3',
 'G#2',
 'F#2',
 'E-3',
 'G#2',
 'C2',
 'F#2',
 'E-3',
 'G#2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'C#3',
 'G#2',
 'F#2',
 'C#3',
 'G#2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'D3',
 'G#2',
 'F#2',
 'E-3',
 'G#2',
 'C2',
 'F#2',
 'E-3',
 'G#2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'C#3',
 'G#2',
 'F#2',
 'C#3',
 'G#2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'D3',
 'G#2',
 'F#2',
 'E-3',
 'G#2',
 'C2',
 'F#2',
 'E-3',
 'G#2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'C#3',
 'G#2',
 'F#2',
 'C#3',
 'G#2',
 'F#2',
 'D3',
 'G#2',
 'D2',
 'F#2',
 'D3',
 'G#2',
 'F#2',
 'G#3',
 'C#3',
 'C2',
 'F#2',
 'G#3',
 'C#3',
 'F#2',
 'G3',
 'C#3',
 'D2',
 'F#2',
 'G3',
 'C#3',
 'D2',
 'F#2',
 'F#3',
 'C#3',
 'F#2

In [6]:
seq_n = 50

pitchnames = sorted(set(item for item in notes))

In [7]:
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print(note_to_int)

{'G#3': 10, 'F#2': 7, 'G#2': 9, 'B-3': 1, 'D2': 4, 'E-3': 6, 'F#3': 8, 'C2': 3, 'C#3': 2, 'D3': 5, 'A3': 0, 'G3': 11}


In [8]:
network_inp = []
network_out = []

for i in range(len(notes) - seq_n):
    seq_in = notes[i:i + seq_n]
    seq_out = notes[i + seq_n]
    network_inp.append([note_to_int[char] for char in seq_in])
    network_out.append(note_to_int[seq_out])

In [9]:
n_patterns = len(network_inp)
n_vocab = len(note_to_int)

In [10]:
network_inp = np.reshape(network_inp, (n_patterns, seq_n, 1))
network_inp = network_inp / float(n_vocab)
print(network_inp.shape)
network_out = np.eye(n_vocab)[network_out]
print(network_out.shape)

(771, 50, 1)
(771, 12)


In [11]:
model = Sequential()
model.add(LSTM(256, input_shape = (network_inp.shape[1], network_inp.shape[2]), 
               return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop')

In [12]:
model.fit(network_inp, network_out, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
771/771 [==============================] - 11s - loss: 2.4281      

In [34]:
start = np.random.randint(0, len(network_inp) - 1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_inp[start]
prediction_output = []

for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

0
1


TypeError: unsupported operand type(s) for /: 'list' and 'float'